# (1) Group Final Report - Group 20 - Calvin Du, Dishika Taneja, Geoff Lei - Stat 301 - Dec. 5, 2025

# (2) Introduction

CALVIN OLD
**Response variable:** `totalbodywatertbw`  

**Explanatory variables:** In stage 2 of Calvin's project he used: `leanmasslm`, `totalbodyfatratiotbfr`, `gender`, `height`, `weight`, `bodymassindexbmi` but we will explore which variables can be used using Lasso Regression.

**Model type:** Multiple Linear Regression

**Focus:** Inference — to identify which body composition features contribute most to totalbodywatertbw

Total Body Water (TBW) is a physiological indicator that reflects how much water is contained in an individuals body. As such, TBW varies with characteristics such as tissue type and overall body size. For example, lean tissue (muscles and organs) contains approximately 70–75% water, whereas fat tissue carries only about 10% (Lorenzo et al., 2019). Basic anthropometric traits such as height, weight, BMI, and gender also relate to TBW because they reflect differences in body size and composition (Ritz et al., 2008).  

Using the variables available in this dataset, we aim to confirm well-established associations, (such as those involving lean mass and body fat ratio, while also exploring the influence of variables that have been less frequently examined, such as gender.
Therefore our central research question is:  

**How do lean mass, gender, body fat ratio, BMI, and other anthropometric measurements individually and jointly explain variation in total body water?** 

To address this question, several multiple linear regression (MLR) models will be fit using totalbodywatertbw as the response variable and differing 
combinations of covariates as predictors. The primary goal is statistical inference about how these variables relate to TBW. In addition, lasso regression will be used to identify which covariates contribute most effectively within this dataset.

# (3) Method and Results

## 3a) Data

In [ ]:
library(tidyverse)
library(broom)
library(glmnet)
library(dplyr)
library(readxl)
library(janitor) 
library(caret)
library(pROC)
library(boot)
library(car)


Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test


Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Type 'citation("pROC")' for a citation.


Attaching package: ‘pROC’


The following objects are masked from ‘package:stats’:

    cov, smooth, var



Attaching package: ‘boot’


The following object is masked from ‘package:lattice’:

    melanoma


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:boot’:

    logit


The following object is masked from ‘package:dplyr’:

    recode


The following object is masked from ‘package:purrr’:

    some




In [ ]:
set.seed(30120)

### Reproducible reading of dataset into R 

In [ ]:
# #Download file from https://archive.ics.uci.edu/dataset/1150/gallstone-1

# # 1. Create a temporary var to hold outer ZIP
# outer_zip  <- tempfile(fileext = ".zip")
# # 2. Download the ZIP directly from UCI, assign to outer_zip variable
# download.file("https://archive.ics.uci.edu/static/public/1150/gallstone-1.zip", outer_zip, mode = "wb")
# # 2. Unzip the outer layer into a temporary directory
# outer_dir <- tempfile()
# unzip(outer_zip, exdir = outer_dir)
# # 3. Unzip the inner ZIP 
# inner_zip <- file.path(outer_dir, "dataset-uci.zip")
# unzip(inner_zip, exdir = outer_dir)
# # 4. Read the Excel file
# excel_path <- file.path(outer_dir, "dataset-uci.xlsx")
# df <- read_excel(excel_path) |> clean_names()

# #Tidiness Checks
# # 1) No duplicate column names
# any(duplicated(names(df)))  # FALSE expected

# # 2) No list-columns 
# all(sapply(df, function(x) is.atomic(x) && !is.list(x)))  # TRUE expected

# # 3) No duplicate rows 
# sum(duplicated(df))  # 0 expected

# # 4) Binary columns truly binary (0/1)
# bin_cols <- c("gallstone_status","gender","coronary_artery_disease_cad",
#               "hypothyroidism","hyperlipidemia","diabetes_mellitus_dm")
# lapply(df[bin_cols], function(x) sort(unique(x)))  # should be c(0,1)
# head(df)

In [ ]:
data <- read.table('dataset-uci.csv', header = TRUE, sep = ",", stringsAsFactors = FALSE)

colnames(data) <- colnames(data) %>%
tolower() %>%
str_replace_all("\\s+", "_") %>%d
str_replace_all("[^a-z0-9_]", "") %>%
str_replace("^([0-9])", "x\\1")
data %>% head()
cat("Column names:\n")
colnames(data)

cat("\nNumber of columns:\n")
print(length(colnames(data)))

cat("\nNumber of rows:\n")
print(nrow(data))

Warning message in file(file, "rt"):
“cannot open file 'dataset-uci.csv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


### Dataset Summary

- **Dataset name:** Gallstone-1
- **Source:** [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/1150/gallstone-1)
- **Number of observations:** 319
- **Number of variables:** 39


| Variable | Description | Type |
|-----------|--------------|------|
| xgallstonestatus | Presence of gallstones | Categorical|
| age | Age | Numeric|
| gender | Gender | Categorical |
| comorbidity | Presence of comorbid diseases | Categorical |
| coronaryarterydiseasecad | Coronary artery disease status | Categorical|
| hypothyroidism | Hypothyroidism status | Categorical|
| hyperlipidemia | Hyperlipidemia status | Categorical|
| diabetesmellitusdm | Diabetes mellitus status | Categorical|
| height, weight, bodymassindexbmi | Anthropometric measures | Numeric |
| totalbodywatertbw, extracellularwaterecw, intracellularwatericw, extracellularfluidtotalbodywaterecftbw | Body water and fluid distribution metrics | Numeric |
| totalbodyfatratiotbfr, leanmasslm, bodyproteincontentprotein, visceralfatratingvfr, bonemassbm, musclemassmm, obesity, totalfatcontenttfc, visceralfatareavfa, visceralmuscleareavmakg | Body composition metrics | Numeric |
| hepaticfataccumulationhfa | Liver fat accumulation indicator | Numeric |
| glucose, totalcholesteroltc, lowdensitylipoproteinldl, highdensitylipoproteinhdl, triglyceride | Blood biochemistry | Numeric |
| aspartataminotransferazast, alaninaminotransferazalt, alkalinephosphatasealp, creatinine, glomerularfiltrationrategfr, creactiveproteincrp, hemoglobinhgb, vitamind | Liver and kidney biomarkers | Numeric |


### Source and Information
- Data were collected prospectively and observationally from patients at Ankara VM Medical Park Hospital (June 2022 – June 2023).

- There were 454 participants but 134 individuals who had gallbladder surgery were omitted -> leading to 319 entries. Then 161 had gallstones and 158 were demmed healthy controls. (Esen et al., 2024).

- Each participant underwent clinical tests including bioimpedance analysis (BIA), blood laboratory tests, and ultrasound screening for gallstones in order to gather features

### Pre-selection of Variables
Some variables are highly correlated or redundant and may be removed. We grouped some of these variables above but here are some more concrete examples: 

- `height` and `weight` are represented by `bodymassindexbmi`

- `totalbodywatertbw`, `extracellularwaterecw`, and `intracellularwatericw` are collinear measures of hydration

- Some prelimary factors showed that variables under binary comorbidity indicators (`coronaryarterydiseasecad`, `diabetesmellitusdm`, etc.) can be summed into a single index as they represent the similar things


## 3b) EDA and Visualization

In [ ]:
df <- read.table('dataset-uci.csv', header = TRUE, sep = ",", stringsAsFactors = FALSE)

colnames(df) <- colnames(df) %>%
tolower() %>%
str_replace_all("\\s+", "_") %>%
str_replace_all("[^a-z0-9_]", "") %>%
str_replace("^([0-9])", "x\\1")

# Convert categorical variables to factors
df <- df %>%
  mutate(
    gender = as.factor(gender),
    comorbidity = as.factor(comorbidity),
    hepaticfataccumulationhfa = as.factor(hepaticfataccumulationhfa)
  )

df$gender <- factor(df$gender,
                  levels = c(0, 1),
                  labels = c("Male", "Female"))

  
df %>% head()

# 3c) Plan for Methods (dishika)  

The focus of our analysis is **Inference**. We want to examine the association between an individual’s total body water (in L) and predictors related to gender and body proportional measurements. 

First, we will use **LASSO** regression for variable selection. This method performs automatic variable selection by shrinking the coefficients of less important features to exactly zero. It will narrow down which variables have the strongest ability to make inference about an individual’s total body water. This creates a more interpretable model that focuses only on the important predictors among the many variables present in the dataset. This also reduces overfitting, especially since we have a large number of potential variables.

- To perform LASSO regression, we will first define the response and the predictor matrix.
- To ensure reproducibility of the cross-validation splits, we will fix a random seed.
- We will then fit a LASSO model with K-fold cross-validation using cv.glmnet, which by default uses 10-fold-cross-validation. Hence, the data are split into 10 folds and for each candidate 𝜆, the model is trained on 9 folds and validated on the remaining fold.
- Using the value of 𝜆 that minimizes the cross-validated error, we will extract the estimated coefficients and will then identify the variables with non-zero coefficients to be used in the final model.

Then, we will fit an additive multiple linear regression model (with lm function) using **totelbodywatertbw** as the response variable and the variables selected by LASSO as the predictors. This will help us examine how the selected variables are associated with an individual’s total body water. 

This method is appropriate because the response variable is continuous, and the goal is to quantify how several body composition variables collectively explain variation in total body water.

**Assumptions for the method:** The assumptions of this method include linear relationships between factors and the response, independent observations, constant variance of residuals, and approximately normally distributed errors.

To check whether the linear regression assumptions will hold, we will make a plot of residuals vs fitted values and a normal QQ plot of the residuals. If the points in the residuals vs fitted values scatterplot are randomly scattered around zero showing no clear pattern, the assumption of linearity will be reasonable and there will be homoscedasticity. Furthermore, in the QQ plot, if the points will lie close to the reference line, it will suggest that the residuals are approximately normally distributed. These diagnostic plots will support the use of a multiple linear regression model for inference on the association between the selected predictors and total body water.

**Potential limitations of the proposed method:** Several predictors (height, weight, BMI, and lean mass) may be highly correlated here which can inflate standard errors and might make it hard to interpret the individual coefficients. Another limitation is that the model assumes linearity; however, physiological relationships may be nonlinear or involve interactions.


# 3c) Plan for Methods (Calvin)  
### Appropiateness
The method I chose is to use a multiple linear regression model to examine how lean mass, total body fat ratio, gender, height, weight, and BMI are associated with a person’s total body water (totalbodywatertbw).

This method is appropriate because the response variable is continuous, and the goal is to quantify how several body composition variables collectively explain variation in total body water.

### Assumptions
The assumptions of this method include linear relationships between factors and the response, independent observations, constant variance of residuals, and approximately normally distributed errors.

### Limitations
A potential limitation is that several predictors (height, weight, BMI, and lean mass) may collinear here which can inflate standard errors and might make it hard to interpret the individual coefficients.

Another limitation is that the model assumes linearity; however, physiological relationships may be nonlinear or involve interactions.

# 3d) Code

### Lasso Regression to Determine Model Fit

In order to determine the best variables to use in our model, we will proceed with utilizing lasso regression to narrow down which variables have the strongest predictive ability.

In [ ]:
y <- df$totalbodywatertbw
X <- model.matrix(totalbodywatertbw ~ ., data = df)[, -1, drop = FALSE]

set.seed(30120)
# get lasso model with cross-validation
lasso <- cv.glmnet(X, y, alpha = 1, standardize = TRUE)

lambda_min <- lasso$lambda.min

# Coefficients
coef_min <- coef(lasso, s = lambda_min)

selected_min <- setdiff(rownames(coef_min)[as.numeric(coef_min) != 0], "(Intercept)")

print(selected_min)

The model selected body composition variables that intuitively strongly associate with Total Body Water. Our final model will include sex (female with male as the baseline), extracellular water (ECW), the extracellular fluid ratio (ECF/TBW), muscle mass (MM), and visceral muscle area (VMA). 

All of these variables do make sense as well as water is primarily stored in muscle tissue so therefore, predictors reflecting muscle quantity (MM, VMA) strongly contribute. Hydration distribution (ECW, ECF/TBW) provides additional information regarding fluid balance and sex also remains significant, as women naturally have lower TBW relative to men due to body-composition differences.

In [ ]:
plot(lasso)
abline(v = log(lasso$lambda.min), col="red", lty=2)

The CV LASSO curve shows that moderate regularization achieves the lowest prediction error. The optimal penalty parameter which is defined as $\lambda_{min}$ yields a model containing five physiologically meaningful predictors of Total Body Water.

In [ ]:
# Model we will use: 
model <- lm(formula = totalbodywatertbw ~ gender + extracellularwaterecw + extracellularfluidtotalbodywaterecftbw + musclemassmm + visceralmuscleareavmakg, data = df)
tidy(summary(model))
summary(model)

# Plots

### Exploring Muscle mass vs Total Body Water by Gender

In [ ]:
ggplot(df, aes(x = musclemassmm, y = totalbodywatertbw, color = gender)) +
  geom_point(alpha = 0.7) +
  geom_smooth(method = "lm", se = FALSE) +
  labs(
    title = "Total Body Water vs Muscle Mass by Gender",
    x = "Muscle Mass (mm)",
    y = "Total Body Water (L)",
    color = "Gender"
  ) 


ERROR while rich displaying an object: Error in `geom_point()`:
! Problem while computing aesthetics.
ℹ Error occurred in the 1st layer.
Caused by error:
! object 'musclemassmm' not found

Traceback:
1. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
2. tryCatchList(expr, classes, parentenv, handlers)
3. tryCatchOne(expr, names, parentenv, handlers[[1L]])
4. doTryCatch(return(expr), name, parentenv, handler)
5. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_h

**Relevancy**: 

This plot is relevant as it let's us explore as someone get's more muscle mass, how is this associated with how much water is in their body. We also want to see how this changes between gender.

**Results:**

We can see a possitive association with muscle mass and total body water which makes sense as more muscle = more water. Additionally, we can see that Males on average have more body water along with more muscle mass.

### Residual diagnostic plots

In [ ]:
plot(model, which = 1)
plot(model, which = 2)


Looking at the diagnostic plots, our model satisfies the assumptions of linear regression. 

Residuals show no major heteroscedasticity or nonlinearity, and the QQ plot confirms approximate normality. These results support the validity of inference for the final model.

# Computational Code Continued

In [ ]:
model <- lm(formula = totalbodywatertbw ~ gender + extracellularwaterecw + extracellularfluidtotalbodywaterecftbw + musclemassmm + visceralmuscleareavmakg, data = df)

summary(model) %>% tidy(conf.int = TRUE)
summary(model)

### Interpretation

We can see that extracellularwaterecw and extracellularfluidtotalbodywaterecftbw have statistical signifiance in our model while the other variables have p-values that suggest less signifiance.

## 3d) Results

The variables selected using LASSO regression were:
- "genderFemale" (female with male as the baseline)
- "extracellularwaterecw" (extracellular water)
- "extracellularfluidtotalbodywaterecftbw" (the extracellular fluid ratio (ECF/TBW))
- "musclemassmm" (muscle mass (MM))
- "visceralmuscleareavmakg" (visceral muscle area (VMA))

Residual vs fitted values plot and normal QQ plots did not show severe violations of linearity, constant variane or normality. Hence, using the variables selected by LASSO, we fitted an additive multiple linear regression model:

$totalbodywatertbw_i = \hat{\beta_0}  + \hat{\beta_1} genderFemale_i+ \hat{\beta_2} extracellularwaterecw_i+ \hat{\beta_3} extracellularfluidtotalbodywaterecftbw + \hat{\beta_4} musclemassmm + \hat{\beta_5}visceralmuscleareavmakg $

After fitting we found the following values,\
$\mathbf{R^2}$ = 0.9383\
**Adjusted $\mathbf{R^2}$** = 0.9374\
This means that about 94% of the variability in total body water is explained by the selected predictors. Hence, we can say that the model is statistically significant. 

**Interpretation of coefficients:**
- $\hat{\beta_0} = 34.44$ is the intercept, which is the estimated mean total body water for a male when all other continuous predictors are 0.
- $\hat{\beta_1} = 0.05$ is the coefficient for genderFemale. Keeping all other predictors constant, females are estimated to have about 0.05L higher total body water than males.
- $\hat{\beta_2} = 2.34$ is the coefficient for extracellularwaterecw. Keeping all other predictors constant, a 1L increase in extracellular water is assoicated with an increase in estimated total body water on average by about 2.34L.
- $\hat{\beta_3} = -0.83$ is the coefficient for extracellularfluidtotalbodywaterecftbw. Keeping all other predictors constant, a 0.1 increase in the extracellular fluid ratio is associated with a 0.83L decrease in the estimated total body water.
- $\hat{\beta_4} = -0.0095$ is the coefficient for musclemassmm. Keeping all other predictors constant, a 1 unit increase in the muscle mass is associated with a 0.0095L decrease in the estimated total body water. The estimated coefficient is very small hence does not appear to have a significant impact on total body water.
- $\hat{\beta_5} = 0.063$ is the coefficient for visceralmuscleareavmakg. Keeping all other predictors constant, a 1 unit increase in the visceral muscle area is associated with a 0.063L increase in the estimated total body water. The value is gain very small and hence does not appear to have a significant impact on total body water.

**Hypothesis tests of interest:**\
$H_0 : \beta_j = 0$ vs $H_1 : \beta_j \ne 0$


**Based on p-values,** at a 5% significance level, we can say that the only statistically significant predictors are extracellularwaterecw (ECW) and extracellularfluidtotalbodywaterecftbw (ECF/TBW ratio) because their p-value < 0.05. Gender, muscle mass and VMA are not statistically significant predictors because their p-values > 0.05.


**Comparison to other models:**
Compared with a full model that includes all predictors, the LASSO selected model is simpler and more interpretable. It is also statistically efficient since less important variables have been removed, hence preventing overfitting. This model also helps us focus on inference, allowing us to state clearly that ECW and ECF/TBW ratio are the most important features associated with Total Body Water. 

Therefore, these results answer our scientific question of understanding the association between an individual's total body water and other predictors. Also, they help us identify which body composition features contribut the most to Total body water. We can say that extracellular water(ECW) and ECF/TBW ratio are the featuers that contribute the most to total body water.

# (4) Discussion

# (5) References

1. Lorenzo I, Santos DA, Cunha G, Nunez-Felipe L, Figueiredo P, Silva AM, et al. The role of water homeostasis in muscle function and frailty. Nutrients. 2019;11(3):E552. 

2. Ritz P, Vol S, Berrut G, Tack I, Arnaud MJ, Tichet J, et al. Influence of gender and body composition on hydration and body water spaces. Clin Nutr. 2008 Oct;27(5):740–746.  